In [26]:
track_data = 'track2-4m-a.txt'

file = [ line.rstrip('\n') for line in open(track_data, 'r').readlines() ]
str=''
for f in file:
    str+=f
str

'...'

In [27]:
# Decode to bit pattern
bs = ''
for i in str:
    v = eval('0x'+i)
    for b in range(4):
        bit = '0' if (v & 0x08)!=0x08 else '1'
        v=v<<1
        bs+=bit
bs

'...'

In [28]:
count = 1
prev = '0'
stream = ''
histo = [ 0 for i in range(40) ]
for c in bs:
    if prev==c:
        count+=1
        continue
    prev = c
    if count>=len(histo): count=len(histo)-1
    histo[count]+=1
    count=1

for i,v in enumerate(histo):
    print(i, v)

0 0
1 0
2 0
3 0
4 1
5 0
6 0
7 0
8 0
9 0
10 3
11 5
12 4
13 14
14 1257
15 10480
16 23784
17 4446
18 3
19 1
20 3
21 15
22 651
23 1725
24 1805
25 2019
26 415
27 3
28 2
29 0
30 19
31 84
32 94
33 54
34 33
35 5
36 0
37 0
38 0
39 7


In [29]:
count = 1
prev = '0'
stream = ''
for c in bs:
    if prev==c:
        count+=1
        continue
    prev = c
    if   count < 19:      # 1M=5 
        stream +=   '01'
    elif count < 28:      # 1M=7
        stream +=  '001'
    else:
        stream += '0001'
    count=1
stream

'...'

In [30]:
read_bit_count = 0
read_data = 0
raw_bit_stream = 0
data_valid = False
missing_clock = False
MFM_count = 0

print(' 00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 14 15 16 17 18 19 1A 1B 1C 1D 1E 1F')
print('---'*32)

for c in stream:
    bit = 1 if c=='1' else 0
    if read_bit_count % 2==1:
        read_data  = read_data<<1 | bit
    raw_bit_stream = (raw_bit_stream<<1 | bit) & 0x7fff
    read_bit_count += 1

    if read_bit_count >= 16:
        data_valid    = True

    if raw_bit_stream==0x5224 or raw_bit_stream==0x4489:
        data_valid    = True
        missing_clock = True

    if data_valid == True:
        MFM_count += 1
        #"""
        if missing_clock == True:
            print('*', end='')
        else:
            print(' ', end='')
        print(format(read_data, '02X'), end='')
        #"""
        """
        if read_data>=ord(' ') and read_data<=ord('Z'):
            print(chr(read_data), end='')
        """
        read_data      = 0
        read_bit_count = 0
        data_valid     = False
        missing_clock  = False
        #raw_bit_stream = 0

        if MFM_count % 32==0:
            print('')


 00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 14 15 16 17 18 19 1A 1B 1C 1D 1E 1F
------------------------------------------------------------------------------------------------
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF F7 CB FC 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21
 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 21 FF FF FF FF FF
 FF FF FF FF FF FF FE*02*02*A1*A1 FE 01 01 04 01 44 7D 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E
 4E 4E 4E 4E 4E 4E 4E 4E 00 00 00 00 00 00 00 00 00 00 00 00 00*14*02*A1*A1 FB FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF

 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FB E5 FE 10 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90
 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90 90
 90 90 FF FF FF FF FF FF FF FF FF FF FF FF*02*02*A1*A1 FE 01 01 0D 01 FE E5 4E 4E 4E 4E 4E 4E 4E
 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 01 FF FF FF FF FF FF FF FF FF FF FF FE*02*02*A1*A1
 FB FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF FF
 FF FF FF FF FF FF FF FF FF FF